<a href="https://colab.research.google.com/github/ilansarbac95/DI158/blob/main/DI_MiniProject_BrazilianEcommerce_SQL_IS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install sqlalchemy==2.0.29
import pandas as pd, sqlite3, zipfile, os, glob, re
from sqlalchemy import create_engine
pd.options.display.max_rows = 50

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 15.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
up = files.upload()
zip_name = next(iter(up))
extract_dir = "/content/olist_zip"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall(extract_dir)
print("Extracted to:", extract_dir)

Saving Brazilian E-Commerce Dive Deep using SQL.zip to Brazilian E-Commerce Dive Deep using SQL.zip
Extracted to: /content/olist_zip


In [ ]:
!pip -q install sqlalchemy==2.0.29
import pandas as pd, sqlite3, zipfile, os, glob, re
pd.options.display.max_rows = 50

In [ ]:
from google.colab import files
up = files.upload()                 # ⬅️ choisis ton ZIP Olist
zip_name = next(iter(up))

extract_dir = "/content/olist_zip"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall(extract_dir)
print("Extracted to:", extract_dir)


Saving Brazilian E-Commerce Dive Deep using SQL.zip to Brazilian E-Commerce Dive Deep using SQL (1).zip
Extracted to: /content/olist_zip


In [ ]:
db_path = "/content/brazilian_ecommerce.sqlite"
if os.path.exists(db_path):
    os.remove(db_path)
conn = sqlite3.connect(db_path)

name_map = {
    "olist_customers_dataset.csv": "customers",
    "olist_geolocation_dataset.csv": "geolocation",
    "olist_order_items_dataset.csv": "order_items",
    "olist_order_payments_dataset.csv": "order_payments",
    "olist_order_reviews_dataset.csv": "order_reviews",
    "olist_orders_dataset.csv": "orders",
    "olist_products_dataset.csv": "products",
    "olist_sellers_dataset.csv": "sellers",
    "product_category_name_translation.csv": "product_category_name_translation",
}

def to_snake(cols):
    return [re.sub(r'[^0-9a-zA-Z_]+','_', c).lower() for c in cols]

loaded = []
# on parcourt TOUT et on ne prend que les vrais fichiers
for csv_name, table in name_map.items():
    csv_path = None
    for root, dirs, files in os.walk(extract_dir):
        for f in files:
            if f == csv_name:
                csv_path = os.path.join(root, f)
                break
        if csv_path: break
    if not csv_path:
        print("⚠️ Introuvable :", csv_name);
        continue

    print(f"→ Charge {table} depuis: {csv_path}")
    df = pd.read_csv(csv_path, encoding="utf-8", low_memory=False)
    df.columns = to_snake(df.columns)
    df.to_sql(table, conn, index=False, if_exists="replace")
    loaded.append((table, len(df)))

# vues alias olist_*
cur = conn.cursor()
for v, t in [
    ("olist_customers","customers"),
    ("olist_orders","orders"),
    ("olist_order_items","order_items"),
    ("olist_order_payments","order_payments"),
    ("olist_order_reviews","order_reviews"),
    ("olist_products","products"),
    ("olist_sellers","sellers"),
    ("olist_geolocation","geolocation"),
]:
    cur.execute(f"DROP VIEW IF EXISTS {v}")
    cur.execute(f"CREATE VIEW {v} AS SELECT * FROM {t}")
conn.commit()

import pandas as pd
pd.DataFrame(loaded, columns=["table","rows"])


→ Charge customers depuis: /content/olist_zip/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_customers_dataset.csv/olist_customers_dataset.csv
→ Charge geolocation depuis: /content/olist_zip/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_geolocation_dataset.csv/olist_geolocation_dataset.csv
→ Charge order_items depuis: /content/olist_zip/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_order_items_dataset.csv/olist_order_items_dataset.csv
→ Charge order_payments depuis: /content/olist_zip/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_order_payments_dataset.csv/olist_order_payments_dataset.csv
→ Charge order_reviews depuis: /content/olist_zip/Brazilian E-Commerce Dive Deep using SQL/Brazilian E-Commerce Dive Deep using SQL/olist_order_reviews_dataset.csv/olist_order_reviews_dataset.csv
→ Charge orders depuis: /content/olist_zi

,table,rows
0,customers,99441
1,geolocation,1000163
2,order_items,112650
3,order_payments,103886
4,order_reviews,99224
5,orders,99441
6,products,32951
7,sellers,3095
8,product_category_name_translation,71


In [ ]:
conn = sqlite3.connect("/content/brazilian_ecommerce.sqlite")

# liste des tables/vues pour vérifier que tout est bien créé
pd.read_sql_query("""
SELECT name, type
FROM sqlite_master
WHERE type in ('table','view')
ORDER BY type, name;
""", conn)

# test du brief
pd.read_sql_query("SELECT * FROM olist_customers LIMIT 5;", conn)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
import sqlite3, pandas as pd
db_path = "/content/brazilian_ecommerce.sqlite"  # ajuste si différent
conn = sqlite3.connect(db_path)


In [ ]:
from google.colab import files

def run_sql(name, sql, save_csv=False):
    df = pd.read_sql_query(sql, conn)
    print(f"== {name} ==")
    display(df.head(20))
    if save_csv:
        out = f"{name}.csv"
        df.to_csv(out, index=False)
        files.download(out)
    return df


In [ ]:
sql = """
WITH jan18 AS (
  SELECT o.order_id
  FROM olist_orders o
  WHERE substr(o.order_purchase_timestamp,1,7)='2018-01'
),
jan18_with_reviews AS (
  SELECT j.order_id, r.review_score
  FROM jan18 j
  LEFT JOIN olist_order_reviews r ON r.order_id = j.order_id
)
SELECT
  COUNT(*) AS jan18_orders,
  SUM(CASE WHEN review_score=5 THEN 1 ELSE 0 END) AS five_star_orders,
  ROUND(100.0 * SUM(CASE WHEN review_score=5 THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_five_star
FROM jan18_with_reviews;
"""
q1 = run_sql("q1_jan2018_five_star", sql, save_csv=False)


== q1_jan2018_five_star ==


,jan18_orders,five_star_orders,pct_five_star
0,7313,4097,56.02


In [ ]:
sql = """
SELECT substr(order_purchase_timestamp,1,4) AS year,
       COUNT(DISTINCT customer_id) AS unique_customers,
       COUNT(*) AS orders
FROM olist_orders
GROUP BY 1
ORDER BY 1;
"""
q2 = run_sql("q2_customer_trend_yoy", sql, save_csv=False)


== q2_customer_trend_yoy ==


,year,unique_customers,orders
0,2016,329,329
1,2017,45101,45101
2,2018,54011,54011


In [ ]:
sql = """
WITH order_amounts AS (
  SELECT oi.order_id, SUM(oi.price + oi.freight_value) AS order_value
  FROM olist_order_items oi
  GROUP BY oi.order_id
)
SELECT o.customer_id,
       ROUND(AVG(oa.order_value),2) AS avg_order_value,
       COUNT(*) AS orders_count,
       ROUND(SUM(oa.order_value),2) AS total_value
FROM olist_orders o
JOIN order_amounts oa ON oa.order_id = o.order_id
GROUP BY o.customer_id
ORDER BY avg_order_value DESC;
"""
q3 = run_sql("q3_aov_by_customer", sql, save_csv=False)


== q3_aov_by_customer ==


,customer_id,avg_order_value,orders_count,total_value
0,1617b1357756262bfa56ab541c47bc16,13664.08,1,13664.08
1,ec5b2ba62e574342386871631fafd3fc,7274.88,1,7274.88
2,c6e2731c5b391845f6800c97401a43a9,6929.31,1,6929.31
3,f48d464a0baaea338cb25f816991ab1f,6922.21,1,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66,1,6726.66
5,05455dfa7cd02f13d132aa7a6a9729c6,6081.54,1,6081.54
6,df55c14d1476a9a3467f131269c2477f,4950.34,1,4950.34
7,e0a2412720e9ea4f26c1ac985f6a7358,4809.44,1,4809.44
8,24bbf5fd2f2e1b359ee7de94defc4a15,4764.34,1,4764.34
9,3d979689f636322c62418b6346b1c6d2,4681.78,1,4681.78


In [ ]:
sql = """
WITH order_amounts AS (
  SELECT oi.order_id, SUM(oi.price + oi.freight_value) AS order_value
  FROM olist_order_items oi
  GROUP BY oi.order_id
),
orders_1618 AS (
  SELECT * FROM olist_orders
  WHERE substr(order_purchase_timestamp,1,4) BETWEEN '2016' AND '2018'
)
SELECT c.customer_city, c.customer_state,
       ROUND(SUM(oa.order_value),2) AS revenue
FROM orders_1618 o
JOIN olist_customers c ON c.customer_id = o.customer_id
JOIN order_amounts oa ON oa.order_id = o.order_id
GROUP BY c.customer_city, c.customer_state
ORDER BY revenue DESC
LIMIT 5;
"""
q4 = run_sql("q4_top5_cities_revenue_2016_2018", sql, save_csv=False)


== q4_top5_cities_revenue_2016_2018 ==


,customer_city,customer_state,revenue
0,sao paulo,SP,2170227.12
1,rio de janeiro,RJ,1154234.02
2,belo horizonte,MG,416733.39
3,brasilia,DF,352305.14
4,curitiba,PR,244739.87


In [ ]:
sql = """
WITH order_amounts AS (
  SELECT oi.order_id, SUM(oi.price + oi.freight_value) AS order_value
  FROM olist_order_items oi
  GROUP BY oi.order_id
),
orders_1618 AS (
  SELECT * FROM olist_orders
  WHERE substr(order_purchase_timestamp,1,4) BETWEEN '2016' AND '2018'
)
SELECT c.customer_state,
       ROUND(SUM(oa.order_value),2) AS revenue,
       COUNT(DISTINCT o.order_id) AS orders
FROM orders_1618 o
JOIN olist_customers c ON c.customer_id = o.customer_id
JOIN order_amounts oa ON oa.order_id = o.order_id
GROUP BY c.customer_state
ORDER BY revenue DESC;
"""
q5 = run_sql("q5_statewise_revenue_2016_2018", sql, save_csv=False)


== q5_statewise_revenue_2016_2018 ==


,customer_state,revenue,orders
0,SP,5921678.12,41375
1,RJ,2129681.98,12762
2,MG,1856161.49,11544
3,RS,885826.76,5432
4,PR,800935.44,4998
5,BA,611506.67,3358
6,SC,610213.60,3612
7,DF,353229.44,2125
8,GO,347706.93,2007
9,ES,324801.91,2025


In [ ]:
sql = """
WITH order_amounts AS (
  SELECT oi.order_id, oi.seller_id, SUM(oi.price + oi.freight_value) AS order_value,
         COUNT(*) AS items_count
  FROM olist_order_items oi
  GROUP BY oi.order_id, oi.seller_id
),
seller_reviews AS (
  SELECT oi.seller_id, r.review_score
  FROM olist_order_items oi
  JOIN olist_order_reviews r ON r.order_id = oi.order_id
),
seller_agg AS (
  SELECT s.seller_id,
         SUM(oa.items_count) AS goods_sold,
         ROUND(SUM(oa.order_value),2) AS revenue,
         COUNT(DISTINCT o.customer_id) AS customers,
         ROUND(AVG(sr.review_score),2) AS avg_review
  FROM order_amounts oa
  JOIN olist_orders o ON o.order_id = oa.order_id
  LEFT JOIN seller_reviews sr ON sr.seller_id = oa.seller_id
  JOIN olist_sellers s ON s.seller_id = oa.seller_id
  GROUP BY s.seller_id
)
SELECT * FROM seller_agg
ORDER BY revenue DESC;
"""
q6 = run_sql("q6_top_sellers_revenue_goods_customers", sql, save_csv=False)


== q6_top_sellers_revenue_goods_customers ==


,seller_id,goods_sold,revenue,customers,avg_review
0,4a3ca9315b744ce9f8e9374361493884,3942208,4.673113e+08,1806,3.80
1,7c67e1448b00f6e969d365cea6b010ab,1864588,3.274463e+08,982,3.35
2,6560211a19b47992c3666cc44a7e94c0,4106660,3.055569e+08,1854,3.91
3,da8622b14eb17ae2831f4ac5b9dab84a,2431968,2.903816e+08,1314,4.07
4,4869f7a5dfa277a7dca6462dcf3b52b2,1327088,2.865875e+08,1132,4.12
5,1f50f920176fa81dab994f9023523100,3730692,2.745468e+08,1404,3.98
6,1025f0e2d44d7041d6cf58b6550e0bfa,2043468,2.473636e+08,915,3.85
7,955fee9216a65b617aa5c0531780ce60,2232011,2.391374e+08,1287,4.05
8,cc419e0650a3c5ba77189a1882b7556a,3214525,2.353529e+08,1706,4.07
9,7a67c85e85bb2ce8582c35f2203ad736,1365386,1.896480e+08,1160,4.23


In [ ]:
sql = """
SELECT c.customer_state,
       COUNT(*) AS delivered_orders,
       SUM(CASE WHEN o.order_delivered_customer_date <= o.order_estimated_delivery_date THEN 1 ELSE 0 END) AS on_time,
       ROUND(100.0 * SUM(CASE WHEN o.order_delivered_customer_date <= o.order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) AS on_time_pct
FROM olist_orders o
JOIN olist_customers c ON c.customer_id = o.customer_id
WHERE o.order_status='delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND o.order_estimated_delivery_date IS NOT NULL
GROUP BY c.customer_state
ORDER BY on_time_pct DESC;
"""
q7 = run_sql("q7_delivery_success_rate_by_state", sql, save_csv=False)


== q7_delivery_success_rate_by_state ==


,customer_state,delivered_orders,on_time,on_time_pct
0,RO,243,236,97.12
1,AC,80,77,96.25
2,AM,145,139,95.86
3,AP,67,64,95.52
4,PR,4923,4677,95.00
5,MG,11354,10717,94.39
6,SP,40494,38107,94.11
7,MT,886,826,93.23
8,DF,2080,1933,92.93
9,RS,5344,4962,92.85


In [ ]:
sql = """
WITH order_payment_main AS (
  SELECT op.order_id,
         MAX(op.payment_value) AS payment_value,
         MIN(op.payment_type) AS payment_type
  FROM olist_order_payments op
  GROUP BY op.order_id
),
order_category AS (
  SELECT oi.order_id, p.product_category_name
  FROM olist_order_items oi
  JOIN olist_products p ON p.product_id = oi.product_id
  GROUP BY oi.order_id, p.product_category_name
),
counts AS (
  SELECT oc.product_category_name, opm.payment_type, COUNT(*) AS orders
  FROM order_category oc
  JOIN order_payment_main opm ON opm.order_id = oc.order_id
  GROUP BY oc.product_category_name, opm.payment_type
),
winners AS (
  SELECT product_category_name, MAX(orders) AS max_orders
  FROM counts
  GROUP BY product_category_name
)
SELECT c.product_category_name, c.payment_type, c.orders
FROM counts c
JOIN winners w
  ON c.product_category_name = w.product_category_name
 AND c.orders = w.max_orders
ORDER BY c.product_category_name;
"""
q8 = run_sql("q8_payment_preference_by_category", sql, save_csv=False)


== q8_payment_preference_by_category ==


,product_category_name,payment_type,orders
0,agro_industria_e_comercio,credit_card,125
1,alimentos,credit_card,328
2,alimentos_bebidas,credit_card,164
3,artes,credit_card,147
4,artes_e_artesanato,credit_card,13
5,artigos_de_festas,credit_card,27
6,artigos_de_natal,credit_card,98
7,audio,credit_card,254
8,automotivo,credit_card,2988
9,bebes,credit_card,2288


In [ ]:
# a) Paires ville-client -> ville-vendeur + volume
pairs_sql = """
SELECT
  c.customer_city AS cust_city, c.customer_state AS cust_state,
  s.seller_city   AS sell_city, s.seller_state   AS sell_state,
  COUNT(DISTINCT o.order_id) AS orders
FROM olist_orders o
JOIN olist_customers c ON c.customer_id = o.customer_id
JOIN olist_order_items oi ON oi.order_id = o.order_id
JOIN olist_sellers s ON s.seller_id = oi.seller_id
GROUP BY 1,2,3,4;
"""
pairs = pd.read_sql_query(pairs_sql, conn)

# b) Centroid (lat,lng) par ville
cent_sql = """
SELECT geolocation_city AS city, geolocation_state AS state,
       AVG(geolocation_lat) AS lat, AVG(geolocation_lng) AS lng
FROM olist_geolocation
GROUP BY geolocation_city, geolocation_state;
"""
cent = pd.read_sql_query(cent_sql, conn)

pairs = pairs.merge(cent.add_prefix("cust_"), left_on=["cust_city","cust_state"],
                    right_on=["cust_city","cust_state"], how="left")
pairs = pairs.merge(cent.add_prefix("sell_"), left_on=["sell_city","sell_state"],
                    right_on=["sell_city","sell_state"], how="left")

import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

pairs["avg_distance_km"] = haversine(pairs["cust_lat"], pairs["cust_lng"],
                                     pairs["sell_lat"], pairs["sell_lng"]).round(2)

pairs_sorted = pairs.sort_values(["avg_distance_km","orders"], ascending=[False, False])
pairs_sorted.head(20)


,cust_city,cust_state,sell_city,sell_state,orders,cust_lat,cust_lng,sell_lat,sell_lng,avg_distance_km
23255,pau d'arco,AL,ibitinga,SP,1,45.065933,9.341528,-21.757321,-48.829744,9474.68
13788,ilha dos valadares,PR,sao bento do sul,SC,1,42.184003,-8.723762,-26.244738,-49.386659,8677.88
25280,porto trombetas,PA,ribeirao preto,SP,2,25.573041,-24.113095,-21.180751,-47.804281,5793.53
35784,vila dos cabanos,PA,praia grande,SP,1,20.459462,-27.750555,-24.021424,-46.461824,5344.87
35786,vila dos cabanos,PA,santo andre,SP,1,20.459462,-27.750555,-23.660031,-46.521966,5310.62
35785,vila dos cabanos,PA,salto,SP,1,20.459462,-27.750555,-23.196763,-47.292606,5296.45
13440,ibiajara,BA,sao paulo,SP,1,13.001420,-23.583939,-23.570860,-46.633242,4777.11
4151,boa vista,RR,bombinhas,SC,1,2.825211,-60.698120,-27.156573,-48.503204,3581.22
4154,boa vista,RR,fazenda rio grande,PR,1,2.825211,-60.698120,-25.655912,-49.310101,3396.21
4170,boa vista,RR,sao jose dos pinhais,PR,1,2.825211,-60.698120,-25.530946,-49.175291,3388.70
